In [2]:
import pandas as pd

data = pd.read_csv('movie_bd_v5.csv', parse_dates=['release_date']) # импортируем данные из файла и сразу конвертируем даты

## 0. Анализ источника данных

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

In [4]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [5]:
data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,2015-03-18,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,2015-12-15,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,2015-04-01,7.3,2015


## Вспомогательные функции

In [49]:
def get_by_max_value(df, column):
    """Возвращает фильм с максимальным значением для Series (колонки)"""
    
    return df.query('{0} == {1}'.format(column, df[column].max()))


def get_by_min_value(df, column):
    """Возвращает фильм с минимальным значением для Series (колонки)"""
    
    return df.query('{0} == {1}'.format(column, df[column].min()))


def get_movies_by_years(df, years):
    """Возвращает фильмы, выпущенные в прокат в определенном году (годах)"""

    return df.query('release_year in @years')
    

def get_exploded_data(df, column, separator='|'):
    """Возвращает копию датасета с разбивкой строковых данных из определенного столбца"""

    exploded_df = df.copy()
    exploded_df[column] = exploded_df[column].str.split(separator)

    return exploded_df.explode(column)

## Предобработка

In [8]:
answers = {} # создадим словарь для ответов

data['profit'] = data.apply(lambda row: row.revenue - row.budget, axis=1) # добавим колонку с прибыльностью фильма
data['release_month'] = data.release_date.dt.month # добавим колонку с месяцем выпуска фильмов

## 1. У какого фильма из списка самый большой бюджет?

In [32]:
answers['1'] = data[data.budget == data.budget.max()].original_title.item()

#### Вариант 2

In [33]:
answers['1'] = get_by_max_value(data, 'budget').original_title.item()

'Pirates of the Caribbean: On Stranger Tides'

## 2. Какой из фильмов самый длительный (в минутах)?

In [34]:
answers['2'] = get_by_max_value(data, 'runtime').original_title.item()

## 3. Какой из фильмов самый короткий (в минутах)?

In [35]:
answers['3']  = data.query('runtime == @data.runtime.min()').original_title.item()

#### Вариант 2

In [36]:
answers['3'] = get_by_min_value(data, 'runtime').original_title.item()

## 4. Какова средняя длительность фильмов?

In [17]:
answers['4'] = round(data.runtime.mean())

## 5. Каково медианное значение длительности фильмов?

In [18]:
answers['5'] = round(data.runtime.median())

## 6. Какой фильм самый прибыльный?

In [37]:
answers['6'] = get_by_max_value(data, 'profit').original_title.item()

## 7. Какой фильм самый убыточный?

In [38]:
answers['7'] = get_by_min_value(data, 'profit').original_title.item()

## 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [21]:
answers['8'] = len(data.query('revenue > budget'))

## 9. Какой фильм оказался самым кассовым в 2008 году?

In [39]:
movies_2008 = get_movies_by_years(data, [2008])
answers['9'] = get_by_max_value(movies_2008, 'revenue').original_title.item()

## 10. Самый убыточный фильм за период с 2012 по 2014 годы (включительно)?

In [40]:
period_movies = get_movies_by_years(data, range(2012, 2015))
answers['10'] = get_by_min_value(period_movies, 'profit').original_title.item()

## 11. Какого жанра фильмов больше всего?

In [44]:
movies_by_genre = get_exploded_data(data, 'genres')
answers['11'] = movies_by_genre.genres.value_counts().index[0]

## 12. Фильмы какого жанра чаще всего становятся прибыльными?

In [48]:
profit_movies = data.query('revenue > budget')
movies_by_genre = get_exploded_data(profit_movies, 'genres')
answers['12'] = movies_by_genre.value_counts('genres').index[0]

## 13. У какого режиссёра самые большие суммарные кассовые сборы?

In [47]:
movies_by_director = get_exploded_data(data, 'director')
answers['13'] = movies_by_director.groupby(['director'])['revenue'].sum().sort_values(ascending=False).index[0]

## 14. Какой режиссер снял больше всего фильмов в стиле Action?

In [50]:
movies_by_genre = get_exploded_data(data, 'genres')
action_directors = movies_by_genre[movies_by_genre.genres == 'Action'].director
answers['14'] = action_directors.str.split('|').explode().value_counts().index[0]

## 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году?

In [51]:
movies_2012 = get_movies_by_years(data, [2012])
movies_by_actor = get_exploded_data(movies_2012, 'cast')
answers['15'] = movies_by_actor.groupby(['cast'])['revenue'].sum().sort_values(ascending=False).index[0]

## 16. Какой актер снялся в большем количестве высокобюджетных фильмов?
#### Примечание: в фильмах, где бюджет выше среднего по данной выборке.

In [52]:
high_budget_movies = data[data.budget > data.budget.mean()]
answers['16'] = get_exploded_data(high_budget_movies, 'cast').value_counts('cast').index[0]

## 17. В фильмах какого жанра больше всего снимался Nicolas Cage?

In [53]:
cage_movies = data[data.cast.str.contains('Nicolas Cage')]
cage_movies_by_genre = get_exploded_data(cage_movies, 'genres')
answers['17'] = cage_movies_by_genre.value_counts('genres').index[0]

## 18. Самый убыточный фильм от Paramount Pictures?

In [54]:
paramount_movies = data[data.production_companies.str.contains('Paramount Pictures')]
answers['18'] = paramount_movies.query('profit == @paramount_movies.profit.min()').original_title.item()

## 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [55]:
answers['19'] = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).index[0]

## 20. Какой самый прибыльный год для студии Warner Bros?

In [56]:
warner_movies = data[data.production_companies.str.contains('Warner Bros')]
answers['20'] = warner_movies.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).index[0]

## 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [57]:
answers['21'] = data.release_month.value_counts().index[0]

## 22. Сколько суммарно вышло фильмов летом (за июнь, июль, август)?

In [58]:
answers['22'] = len(data.query('release_month in [6, 7, 8]').index)

## 23. Для какого режиссера зима — самое продуктивное время года?

In [59]:
winter_movies = data.query('release_month in [1, 2, 12]')
winter_movies_by_director = get_exploded_data(winter_movies, 'director')
answers['23'] = winter_movies_by_director.director.value_counts().index[0]

## 24. Какая студия даёт самые длинные названия своим фильмам по количеству символов?

In [60]:
titles_df = data.copy()[['production_companies', 'original_title']]
titles_df['title_length'] = titles_df.original_title.str.len()
titles_by_companies = get_exploded_data(titles_df, 'production_companies')
answers['24'] = titles_by_companies.groupby('production_companies')['title_length'].mean().sort_values(ascending=False).index[0]

## 25. Описания фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
overview_df = data.copy()[['production_companies', 'overview']]
overview_df['overview_length'] = overview_df.overview.apply(lambda x: len(x.split(' '))) # считаем количество слов в колонке с описанием
overview_by_companies = get_exploded_data(overview_df, 'production_companies')
answers['25'] = overview_by_companies.groupby('production_companies')['overview_length'].mean().sort_values(ascending=False).index[0]

## 26. Какие фильмы входят в один процент лучших по рейтингу?

In [70]:
answers['26'] = data[data.vote_average > data.vote_average.quantile(0.99)].original_title


## 27. Какие актеры чаще всего снимаются в одном фильме вместе?

In [71]:
from collections import Counter
from itertools import combinations

actor_pairs = Counter()

for row in data.itertuples(name=None, index=False):
    movie_actors = sorted(row[4].split('|'))

    for pair in list(combinations(movie_actors, 2)):
        actor_pairs[pair] += 1

answers['27'] = actor_pairs.most_common(5)





